In [ ]:
# import argparse

# parser = argparse.ArgumentParser()
# parser.add_argument('--n_epochs', type=int, default=50, help='number of epochs to train')
# parser.add_argument('--batch_size', type=int, default=64, help='batch size for training')
# # 添加更多参数

# # 使用parse_known_args()方法
# args, unknown = parser.parse_known_args()

# # 使用解析后的参数
# n_epochs = args.n_epochs
# batch_size = args.batch_size
# # 使用更多解析后的参数


In [ ]:
# 克隆 PyTorch-GAN 仓库
!git clone https://github.com/eriklindernoren/PyTorch-GAN /content/drive/MyDrive/Colab\ Notebooks/PyTorch-GAN

# 克隆 PyTorch-VAE 仓库
!git clone https://github.com/AntixK/PyTorch-VAE /content/drive/MyDrive/Colab\ Notebooks/PyTorch-VAE

Cloning into '/content/drive/MyDrive/Colab Notebooks/PyTorch-GAN'...
remote: Enumerating objects: 1283, done.
remote: Total 1283 (delta 0), reused 0 (delta 0), pack-reused 1283
Receiving objects: 100% (1283/1283), 68.04 MiB | 12.90 MiB/s, done.
Resolving deltas: 100% (751/751), done.
Updating files: 100% (90/90), done.
Cloning into '/content/drive/MyDrive/Colab Notebooks/PyTorch-VAE'...
remote: Enumerating objects: 859, done.
remote: Total 859 (delta 0), reused 0 (delta 0), pack-reused 859
Receiving objects: 100% (859/859), 46.47 MiB | 12.26 MiB/s, done.
Resolving deltas: 100% (619/619), done.
Updating files: 100% (120/120), done.


In [4]:
!pip install torch torchvision matplotlib scikit-image

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [6]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from torchvision.utils import save_image
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim
from sklearn.manifold import TSNE

# Change directory to the DCGAN folder and import required classes
os.chdir('/content/drive/MyDrive/Colab Notebooks/PyTorch-GAN/implementations/dcgan')
from dcgan import Generator as DCGAN_Generator
from dcgan import Discriminator as DCGAN_Discriminator

usage: colab_kernel_launcher.py [-h] [--n_epochs N_EPOCHS] [--batch_size BATCH_SIZE] [--lr LR]
                                [--b1 B1] [--b2 B2] [--n_cpu N_CPU] [--latent_dim LATENT_DIM]
                                [--img_size IMG_SIZE] [--channels CHANNELS]
                                [--sample_interval SAMPLE_INTERVAL]
colab_kernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-390a0a63-e3b6-4d6c-ba05-177488297f63.json


SystemExit: 2

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
from torch import nn, optim
from time import time
from sklearn.manifold import TSNE

# Load data
dataloader = load_data()

# Initialize models
dcgan_generator = DCGAN_Generator()
vae = VAE(in_channels=CHANNELS, latent_dim=LATENT_DIM, hidden_dims=[32, 64, 128, 256, 512])

# Define optimizers and loss function
optimizer_gan = optim.Adam(dcgan_generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_vae = optim.Adam(vae.parameters(), lr=0.001)
criterion = nn.BCELoss()

# Training loop for GAN
def train_gan(generator, optimizer, dataloader, epochs):
    start_time = time()
    for epoch in range(epochs):
        for i, (imgs, _) in enumerate(dataloader):
            real_imgs = imgs
            valid = torch.ones(imgs.size(0), 1, requires_grad=False)
            fake = torch.zeros(imgs.size(0), 1, requires_grad=False)

            optimizer.zero_grad()
            noise = torch.randn(imgs.size(0), LATENT_DIM)
            gen_imgs = generator(noise)
            g_loss = criterion(discriminator(gen_imgs), valid)
            g_loss.backward()
            optimizer.step()
    training_time = time() - start_time
    return gen_imgs, training_time

# Training loop for VAE
def train_vae(model, optimizer, dataloader, epochs):
    start_time = time()
    for epoch in range(epochs):
        for i, (imgs, _) in enumerate(dataloader):
            optimizer.zero_grad()
            recon_imgs, mu, log_var = model(imgs)
            recon_loss = torch.nn.functional.binary_cross_entropy(recon_imgs, imgs, reduction='sum')
            kld_loss = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
            loss = recon_loss + kld_loss
            loss.backward()
            optimizer.step()
    training_time = time() - start_time
    return recon_imgs, training_time

# Train models and get generated data
gen_imgs_gan, gan_time = train_gan(dcgan_generator, optimizer_gan, dataloader, EPOCHS)
gen_imgs_vae, vae_time = train_vae(vae, optimizer_vae, dataloader, EPOCHS)

# Save generated images
save_image(gen_imgs_gan.data[:25], 'gan_generated.png', nrow=5, normalize=True)
save_image(gen_imgs_vae.data[:25], 'vae_generated.png', nrow=5, normalize=True)

# Visualize using t-SNE
def visualize_tsne(real_imgs, gen_imgs_gan, gen_imgs_vae):
    tsne = TSNE(n_components=2, perplexity=30, n_iter=300)
    real_imgs_flat = real_imgs.view(real_imgs.size(0), -1)
    gen_imgs_gan_flat = gen_imgs_gan.view(gen_imgs_gan.size(0), -1)
    gen_imgs_vae_flat = gen_imgs_vae.view(gen_imgs_vae.size(0), -1)

    all_imgs = torch.cat((real_imgs_flat, gen_imgs_gan_flat, gen_imgs_vae_flat), 0)
    tsne_results = tsne.fit_transform(all_imgs.detach().numpy())

    plt.figure(figsize=(10, 7))
    plt.scatter(tsne_results[:real_imgs.size(0), 0], tsne_results[:real_imgs.size(0), 1], label='Real')
    plt.scatter(tsne_results[real_imgs.size(0):real_imgs.size(0)+gen_imgs_gan.size(0), 0], tsne_results[real_imgs.size(0):real_imgs.size(0)+gen_imgs_gan.size(0), 1], label='GAN')
    plt.scatter(tsne_results[real_imgs.size(0)+gen_imgs_gan.size(0):, 0], tsne_results[real_imgs.size(0)+gen_imgs_gan.size(0):, 1], label='VAE')
    plt.legend()
    plt.title('t-SNE visualization of real and generated images')
    plt.show()

# Get real images for visualization
real_imgs, _ = next(iter(dataloader))
visualize_tsne(real_imgs, gen_imgs_gan, gen_imgs_vae)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Function to calculate FID score
def calculate_fid(real_images, generated_images):
    from scipy.linalg import sqrtm
    import numpy as np

    # Calculate the mean and covariance of real and generated images
    mu_real, sigma_real = np.mean(real_images, axis=0), np.cov(real_images, rowvar=False)
    mu_gen, sigma_gen = np.mean(generated_images, axis=0), np.cov(generated_images, rowvar=False)

    # Calculate FID
    ssdiff = np.sum((mu_real - mu_gen)**2.0)
    covmean = sqrtm(sigma_real.dot(sigma_gen))
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    fid = ssdiff + np.trace(sigma_real + sigma_gen - 2.0 * covmean)
    return fid

# Prepare data for FID calculation
real_imgs_flat = real_imgs.view(real_imgs.size(0), -1).detach().numpy()
gen_imgs_gan_flat = gen_imgs_gan.view(gen_imgs_gan.size(0), -1).detach().numpy()
gen_imgs_vae_flat = gen_imgs_vae.view(gen_imgs_vae.size(0), -1).detach().numpy()

# Calculate FID
fid_gan = calculate_fid(real_imgs_flat, gen_imgs_gan_flat)
fid_vae = calculate_fid(real_imgs_flat, gen_imgs_vae_flat)

# Function to calculate PSNR
def calculate_psnr(real_images, generated_images):
    psnr_values = [psnr(real_images[i], generated_images[i]) for i in range(len(real_images))]
    return np.mean(psnr_values)

# Function to calculate SSIM
def calculate_ssim(real_images, generated_images):
    ssim_values = [ssim(real_images[i].numpy().transpose(1, 2, 0), generated_images[i].numpy().transpose(1, 2, 0), multichannel=True) for i in range(len(real_images))]
    return np.mean(ssim_values)

# Calculate PSNR and SSIM
psnr_gan = calculate_psnr(real_imgs, gen_imgs_gan)
ssim_gan = calculate_ssim(real_imgs, gen_imgs_gan)
psnr_vae = calculate_psnr(real_imgs, gen_imgs_vae)
ssim_vae = calculate_ssim(real_imgs, gen_imgs_vae)

print(f'FID Score for GAN: {fid_gan}')
print(f'FID Score for VAE: {fid_vae}')
print(f'PSNR for GAN: {psnr_gan}')
print(f'PSNR for VAE: {psnr_vae}')
print(f'SSIM for GAN: {ssim_gan}')
print(f'SSIM for VAE: {ssim_vae}')

# Summarize and visualize results
def summarize_results(training_time_gan, training_time_vae, fid_gan, fid_vae, psnr_gan, psnr_vae, ssim_gan, ssim_vae):
    results = {
        'Model': ['GAN', 'VAE'],
        'Training Time (s)': [training_time_gan, training_time_vae],
        'FID Score': [fid_gan, fid_vae],
        'PSNR': [psnr_gan, psnr_vae],
        'SSIM': [ssim_gan, ssim_vae]
    }

    df_results = pd.DataFrame(results)
    print(df_results)

    df_results.plot(x='Model', y=['Training Time (s)', 'FID Score', 'PSNR', 'SSIM'], kind='bar', subplots=True, layout=(2, 2), figsize=(14, 10))
    plt.show()

summarize_results(gan_time, vae_time, fid_gan, fid_vae, psnr_gan, psnr_vae, ssim_gan, ssim_vae)


In [4]:
# !git clone https://github.com/NVlabs/stylegan2

# import os
# os.chdir('/content/stylegan2')
# from stylegan2 import model

fatal: destination path 'stylegan2' already exists and is not an empty directory.


ImportError: cannot import name 'model' from 'stylegan2' (unknown location)

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Load the dataset
def load_data(file_path):
    df = pd.read_csv(file_path)
    return df

# Preprocess the data
def preprocess_data(df):
    df = df.drop(columns=['id', 'Unnamed: 32'])
    df['diagnosis'] = df['diagnosis'].apply(lambda x: 1 if x == 'M' else 0)
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(df)
    return data_scaled

# Build a simple GAN
class GAN:
    def __init__(self, data_dim):
        self.data_dim = data_dim
        self.noise_dim = 100
        self.generator = self.build_generator()
        self.discriminator = self.build_discriminator()
        self.gan = self.build_gan()

    def build_generator(self):
        model = models.Sequential()
        model.add(layers.Dense(128, input_dim=self.noise_dim, activation='relu'))
        model.add(layers.Dense(self.data_dim, activation='tanh'))
        return model

    def build_discriminator(self):
        model = models.Sequential()
        model.add(layers.Dense(128, input_dim=self.data_dim, activation='relu'))
        model.add(layers.Dense(1, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam')
        return model

    def build_gan(self):
        self.discriminator.trainable = False
        model = models.Sequential()
        model.add(self.generator)
        model.add(self.discriminator)
        model.compile(loss='binary_crossentropy', optimizer='adam')
        return model

    def train(self, data, epochs=10000, batch_size=64):
        for epoch in range(epochs):
            # Train Discriminator
            idx = np.random.randint(0, data.shape[0], batch_size)
            real_data = data[idx]
            noise = np.random.normal(0, 1, (batch_size, self.noise_dim))
            fake_data = self.generator.predict(noise)
            d_loss_real = self.discriminator.train_on_batch(real_data, np.ones((batch_size, 1)))
            d_loss_fake = self.discriminator.train_on_batch(fake_data, np.zeros((batch_size, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Train Generator
            noise = np.random.normal(0, 1, (batch_size, self.noise_dim))
            g_loss = self.gan.train_on_batch(noise, np.ones((batch_size, 1)))

            # Print progress
            if epoch % 1000 == 0:
                print(f"{epoch} [D loss: {d_loss}] [G loss: {g_loss}]")

# Variational Autoencoder (VAE)
class VAE:
    def __init__(self, data_dim):
        self.data_dim = data_dim
        self.latent_dim = 2
        self.encoder = self.build_encoder()
        self.decoder = self.build_decoder()
        self.vae = self.build_vae()

    def build_encoder(self):
        inputs = layers.Input(shape=(self.data_dim,))
        h = layers.Dense(128, activation='relu')(inputs)
        z_mean = layers.Dense(self.latent_dim)(h)
        z_log_var = layers.Dense(self.latent_dim)(h)
        z = layers.Lambda(self.sampling, output_shape=(self.latent_dim,))([z_mean, z_log_var])
        return models.Model(inputs, [z_mean, z_log_var, z])

    def build_decoder(self):
        inputs = layers.Input(shape=(self.latent_dim,))
        h = layers.Dense(128, activation='relu')(inputs)
        outputs = layers.Dense(self.data_dim, activation='sigmoid')(h)
        return models.Model(inputs, outputs)

    def build_vae(self):
        inputs = layers.Input(shape=(self.data_dim,))
        z_mean, z_log_var, z = self.encoder(inputs)
        outputs = self.decoder(z)
        vae = models.Model(inputs, outputs)
        reconstruction_loss = tf.keras.losses.mse(inputs, outputs)
        reconstruction_loss *= self.data_dim
        kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
        kl_loss = tf.reduce_sum(kl_loss, axis=-1)
        kl_loss *= -0.5
        vae_loss = tf.reduce_mean(reconstruction_loss + kl_loss)
        vae.add_loss(vae_loss)
        vae.compile(optimizer='adam')
        return vae

    def sampling(self, args):
        z_mean, z_log_var = args
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

    def train(self, data, epochs=50, batch_size=64):
        self.vae.fit(data, data, epochs=epochs, batch_size=batch_size)

# Load and preprocess the data
file_path = '/content/drive/MyDrive/deep_generative_models_project/data.csv'
data = load_data(file_path)
data_scaled = preprocess_data(data)

# Train GAN
gan = GAN(data_scaled.shape[1])
gan.train(data_scaled)

# Train VAE
vae = VAE(data_scaled.shape[1])
vae.train(data_scaled)

# Generate synthetic data
noise = np.random.normal(0, 1, (1, gan.noise_dim))
synthetic_data_gan = gan.generator.predict(noise)

synthetic_data_vae = vae.decoder.predict(np.random.normal(size=(1, vae.latent_dim)))

print("Synthetic data generated by GAN:", synthetic_data_gan)
print("Synthetic data generated by VAE:", synthetic_data_vae)


流式输出内容被截断，只能显示最后 5000 行内容。
2/2 [==============================] - 0s 13ms/step
6000 [D loss: 0.3766428381204605] [G loss: 1.3395875692367554]
2/2 [==============================] - 0s 7ms/step
7000 [D loss: 0.2959839999675751] [G loss: 1.4334664344787598]
2/2 [==============================] - 0s 10ms/step
8000 [D loss: 0.37460261583328247] [G loss: 1.5576928853988647]
2/2 [==============================] - 0s 7ms/step
9000 [D loss: 0.2831692397594452] [G loss: 1.4849251508712769]
2/2 [==============================] - 0s 11ms/step
Epoch 1/50
9/9 [==============================] - 1s 7ms/step - loss: 38.9768
Epoch 2/50
9/9 [==============================] - 0s 6ms/step - loss: 38.1130
Epoch 3/50
9/9 [==============================] - 0s 6ms/step - loss: 36.9323
Epoch 4/50
9/9 [==============================] - 0s 9ms/step - loss: 35.5076
Epoch 5/50
9/9 [==============================] - 0s 8ms/step - loss: 33.1790
Epoch 6/50
9/9 [==============================] - 0s 7ms/step - loss: 31